In [23]:
import os
import numpy as np # linear algebra
import pandas as pd

In [24]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import cv2
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Flatten,Dropout,Conv2D,MaxPooling2D,BatchNormalization

TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [ ]:
imre = plt.imread("data_2/leaf_image/bacterial_blight_disease/BACTERIALBLIGHT_001.jpg")
img = plt.imshow(imre)

In [25]:
imre.shape

(300, 300, 3)

In [26]:
label = []
image = []
image_width = 500
image_height = 500

# First loop for label names
# secound lopp for image names
for i in os.listdir("data_2/leaf_image"):
        for j in  os.listdir("data_2/leaf_image/"+i):
            img = cv2.imread("data_2/leaf_image/"+i+"/"+j)
            img = cv2.resize(img,(image_width,image_height))
            label.append(i)
            image.append(img)

x = np.array(image)
y = np.array(label)

x.shape , y.shape

((450, 500, 500, 3), (450,))

In [27]:
encode = LabelEncoder()
y = encode.fit_transform(y)

In [28]:
x_train , x_test, y_train,y_test = train_test_split(x,y,test_size=0.2,stratify=y,random_state=10)

In [32]:
# Scale value
x_train = x_train/255
x_test = x_test/255

In [33]:
# Data Augmentation
data_agu = keras.Sequential([
            keras.layers.experimental.preprocessing.RandomFlip(),
            keras.layers.experimental.preprocessing.RandomRotation(0.4),
            keras.layers.experimental.preprocessing.RandomContrast(0.4)
            ])

NameError: name 'keras' is not defined

In [ ]:
# Orignal Image
plt.subplot(1,2,1)
plt.imshow(x[0])
plt.title(label[0])

# Data Agumentataion Applayed Image
plt.subplot(1,2,2)
plt.imshow(data_agu(x)[0])

In [49]:
model = Sequential([
    data_agu,
    Conv2D(16, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(200, activation='relu'),
    Dense(4)
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
epoch=150
history = model.fit(x_train[:80],y_train[:80],epochs=epoch,verbose=0, validation_data=(x_train[80:],y_train[80:]))

In [ ]:
model.evaluate(x_test,y_test)

In [ ]:
history.history.keys()

In [ ]:
loss = history.history["loss"]
val_loss = history.history["val_loss"]
accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]

plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
plt.plot(range(epoch),loss,label="Traning Loss")
plt.plot(range(epoch),val_loss,label="Validation Loss")
plt.title("Loss")
plt.legend()

plt.subplot(1,2,2)
plt.plot(range(epoch),accuracy,label="Traning Accuracy")
plt.plot(range(epoch),val_accuracy,label="Validation Accuracy")
plt.title("Accuracy")
plt.legend()

Prédiction


In [48]:
from skimage import io

urls = ["https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRz0zUAIvitgPs4-0zsFdH1gzJqZ_6z8YCOjg&usqp=CAU",
        "data/test/DSC_0365.jpg"
       ]

for i in urls:
    try_image = io.imread(i)
    try_image = cv2.resize(try_image,(image_width,image_height))
    try_image = try_image/255
    plt.imshow(try_image)
    plt.show()
    try_image = try_image.reshape(1,image_width,image_height,3)

    # If Png Image , that is 4 channel image so convert into 3 channel
    #convert the image from RGBA2RGB
    if len(try_image.shape) > 2 and try_image.shape[2] == 4:
        q = cv2.cvtColor(try_image, cv2.COLOR_BGRA2BGR)
        pridiction = np.argmax(model.predict([q]))

    else:
        pridiction = np.argmax(model.predict([try_image]))

    print("Pridicted Label ",pridiction)
    print(encode.inverse_transform([pridiction]))

PermissionError: [WinError 32] Le processus ne peut pas accéder au fichier car ce fichier est utilisé par un autre processus: 'C:\\Users\\user\\AppData\\Local\\Temp\\tmp89o4jauy'